<div style=" border-bottom: 8px solid #e3f56c; overflow: hidden; border-radius: 10px; height: 95%; width: 100%; display: flex;">
  <div style="height: 100%; width: 100%; background-color: #3800BB; float: left; text-align: center; display: flex; justify-content: left; align-items: center; font-size: 40px; ">
    <b><span style="color: #FFFFFF; padding: 20px 20px;">Santander Customer Satisfaction: draft</span></b>
  </div>
</div>

In [5]:
import sys
import os
# Add the parent directory to the Python path
sys.path.append(os.path.abspath('..'))


from classes import Paths
import src.data_utils as data_utils
import pandas as pd
pd.set_option("display.float_format", "{:,.2f}".format)
pd.set_option("display.max_columns", None)
import numpy as np
import warnings
warnings.filterwarnings("ignore")


from src.data_utils import setup_logging
setup_logging()

In [6]:
paths = Paths()
# Load the data
df_train = data_utils.load_data(paths.get_processed_data_path("train_cleaned.csv"))
df_test = data_utils.load_data(paths.get_processed_data_path("test_cleaned.csv"))
df_train.head()

,num_var22_hace3,num_var40_0,delta_imp_reemb_var17_1y3,num_op_var39_efect_ult3,num_meses_var12_ult3,num_op_var39_ult1,saldo_medio_var29_ult1,delta_num_reemb_var33_1y3,imp_venta_var44_ult1,saldo_medio_var5_ult1,num_var45_hace2,num_op_var39_ult3,imp_aport_var17_ult1,saldo_medio_var5_hace2,num_op_var40_comer_ult3,saldo_var31,num_var17,ind_var37,imp_var7_recib_ult1,saldo_medio_var13_corto_hace2,imp_aport_var33_hace3,num_var5_0,delta_num_aport_var13_1y3,imp_aport_var33_ult1,num_var8,num_var32,delta_num_aport_var17_1y3,num_aport_var17_hace3,num_op_var41_efect_ult1,ind_var25,saldo_var12,num_meses_var8_ult3,num_var25,ind_var25_cte,num_op_var40_hace3,imp_op_var39_efect_ult1,num_var13_largo,num_var43_recib_ult1,saldo_medio_var29_hace3,imp_op_var41_comer_ult1,num_var37_med_ult2,saldo_medio_var44_ult1,imp_var43_emit_ult1,num_var4,var3,ind_var10_ult1,delta_num_compra_var44_1y3,saldo_var13_largo,num_op_var41_comer_ult1,delta_imp_trasp_var17_out_1y3,imp_op_var40_efect_ult1,num_op_var39_comer_ult3,saldo_var6,ind_var13_corto,num_var24_0,num_venta_var44_ult1,imp_op_var39_efect_ult3,num_sal_var16_ult1,saldo_var13_medio,num_compra_var44_ult1,num_op_var40_hace2,imp_aport_var13_hace3,imp_trasp_var17_in_hace3,num_var45_hace3,saldo_medio_var33_hace2,ind_var5,num_trasp_var11_ult1,imp_op_var39_ult1,delta_imp_trasp_var17_in_1y3,num_op_var40_efect_ult3,ind_var9_cte_ult1,num_var7_recib_ult1,imp_trasp_var33_in_ult1,saldo_var18,saldo_medio_var8_hace2,num_var42_0,ind_var30,imp_venta_var44_hace3,imp_trans_var37_ult1,num_op_var39_hace2,saldo_medio_var44_hace3,delta_imp_trasp_var33_out_1y3,num_meses_var13_largo_ult3,num_op_var40_efect_ult1,saldo_var32,num_med_var22_ult3,num_var20,saldo_var30,imp_reemb_var17_ult1,ind_var5_0,saldo_medio_var13_corto_ult1,saldo_medio_var12_hace3,saldo_medio_var13_corto_hace3,num_var39,num_var30,imp_trasp_var33_in_hace3,delta_imp_reemb_var13_1y3,saldo_medio_var8_ult1,imp_trasp_var17_in_ult1,saldo_var26,var15,num_var30_0,num_var37,num_aport_var17_ult1,saldo_medio_var8_ult3,num_var44_0,saldo_medio_var12_hace2,imp_reemb_var13_ult1,var21,num_var24,num_op_var41_hace3,ind_var43_recib_ult1,imp_op_var40_comer_ult1,num_var22_hace2,num_aport_var13_hace3,num_meses_var39_vig_ult3,delta_imp_aport_var33_1y3,num_var43_emit_ult1,num_var42,num_var14,num_op_var40_ult1,delta_imp_amort_var34_1y3,var36,saldo_medio_var33_ult1,saldo_var40,saldo_medio_var44_hace2,saldo_medio_var8_hace3,imp_var7_emit_ult1,saldo_var44,imp_op_var41_comer_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var33_hace3,saldo_var8,imp_reemb_var17_hace3,saldo_var13,num_aport_var13_ult1,num_med_var45_ult3,delta_imp_amort_var18_1y3,saldo_medio_var29_hace2,delta_num_trasp_var33_in_1y3,num_var44,saldo_var37,var38,imp_op_var40_efect_ult3,saldo_var5,ind_var37_cte,imp_op_var40_comer_ult3,saldo_var14,num_var31_0,imp_sal_var16_ult1,num_var41_0,num_var22_ult1,num_var12_0,ind_var14_0,delta_num_venta_var44_1y3,num_var13_0,ind_var13,ind_var12_0,num_ent_var16_ult1,imp_aport_var13_ult1,saldo_medio_var13_largo_ult1,imp_op_var40_ult1,imp_trasp_var17_out_ult1,imp_ent_var16_ult1,ind_var13_largo_0,ind_var39_0,saldo_var34,ind_var43_emit_ult1,num_var20_0,num_meses_var5_ult3,saldo_medio_var5_ult3,saldo_medio_var13_largo_hace3,num_var17_0,saldo_medio_var13_largo_hace2,num_var45_ult1,num_var8_0,num_var33_0,num_op_var40_comer_ult1,imp_compra_var44_hace3,saldo_medio_var5_hace3,saldo_var20,ID,num_var31,num_var22_ult3,num_var14_0,TARGET
0,0,0,0,0,0,0,0.00,0,0.00,0.00,0,0,0.00,0.00,0,0.00,0,0,0.00,0.00,0,3,0.00,0,0,0,0.00,0,0,0,0.00,0,0,0,0,0.00,0,0,0.00,0.00,0,0.00,0.00,0,2,0,0.00,0.00,0,0,0.00,0,0.00,0,0,0,0.00,0,0,0,0,0.00,0.00,0,0.00,0,0,0.00,0,0,0,0,0.00,0,0.00,3,0,0.00,0.00,0,0.00,0,0,0,0.00,0,0,0.00,0.00,1,0.00,0.00,0.00,0,0,0.00,0,0.00,0.00,0.00,23,3,0,0,0.00,0,0.00,0.00,0,0,0,0,0.00,0,0,2,0.00,0,0,0,0,0,99,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0.00,0,0,0.00,"39,205.17",0.00,0.00,0,0.00,0.00,0,0.00,3,0,0,0,0.00,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0,1,0,0,0,0,0.00,0.00,0,0.00,0,0,0,

In [7]:
# shape
df_train.shape

df_test.shape

(75818, 185)

In [29]:
df_train.__assigned_df_name__ = "df_train"
df_test.__assigned_df_name__ = "df_test"

for i_df in [df_train, df_test]:
    tmp_unique_dtypes = []
    for col in i_df.columns:
        tmp_unique_dtypes.append(str(i_df[col].dtype))
    print(f"Shape of {i_df.__assigned_df_name__}: {i_df.shape}")  # shape
    print("Missing values:", i_df.isna().any().sum())   # missing values
    print(f"Data types in {i_df.__assigned_df_name__}: {set(tmp_unique_dtypes)}")

del i_df, tmp_unique_dtypes

Shape of df_train: (76020, 186)
Missing values: 0
Data types in df_train: {'int64', 'float64'}
Shape of df_test: (75818, 185)
Missing values: 0
Data types in df_test: {'int64', 'float64'}


<div class="alert" style="background-color: #FEF9E7; border-left: 8px solid #D4AC0D; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

Dataframes have no missing values and datatypes are int64 and float64
</div>

In [36]:
def find_categorical_candidates(df_train, threshold=0.05, max_unique_values=50):
    n_rows = df_train.shape[0]
    categorical_candidates = []

    for col in df_train.columns:
        if df_train[col].dtype in ['int64', 'float64']:
            n_unique = df_train[col].nunique(dropna=True)
            unique_ratio = n_unique / n_rows

            if n_unique <= max_unique_values or unique_ratio < threshold:
                categorical_candidates.append(col)

    return categorical_candidates

# Find possible categorical columns
categorical_cols = find_categorical_candidates(df_train)
print("Number of possible categorical columns:", len(categorical_cols))
print("Possible categorical columns:", categorical_cols)


for i in [0.001, 0.002, 0.003, 0.004, 0.005]:
    categorical_cols = find_categorical_candidates(df_train, threshold=i)
    print(f"Threshold: {i}, Number of possible categorical columns: {len(categorical_cols)}")
    print("Possible categorical columns:", categorical_cols)

Number of possible categorical columns: 173
Possible categorical columns: ['num_var22_hace3', 'num_var40_0', 'delta_imp_reemb_var17_1y3', 'num_op_var39_efect_ult3', 'num_meses_var12_ult3', 'num_op_var39_ult1', 'saldo_medio_var29_ult1', 'delta_num_reemb_var33_1y3', 'imp_venta_var44_ult1', 'num_var45_hace2', 'num_op_var39_ult3', 'imp_aport_var17_ult1', 'num_op_var40_comer_ult3', 'saldo_var31', 'num_var17', 'ind_var37', 'imp_var7_recib_ult1', 'saldo_medio_var13_corto_hace2', 'imp_aport_var33_hace3', 'num_var5_0', 'delta_num_aport_var13_1y3', 'imp_aport_var33_ult1', 'num_var8', 'num_var32', 'delta_num_aport_var17_1y3', 'num_aport_var17_hace3', 'num_op_var41_efect_ult1', 'ind_var25', 'saldo_var12', 'num_meses_var8_ult3', 'num_var25', 'ind_var25_cte', 'num_op_var40_hace3', 'imp_op_var39_efect_ult1', 'num_var13_largo', 'num_var43_recib_ult1', 'saldo_medio_var29_hace3', 'num_var37_med_ult2', 'saldo_medio_var44_ult1', 'imp_var43_emit_ult1', 'num_var4', 'var3', 'ind_var10_ult1', 'delta_num_compr

In [33]:
len(categorical_cols)

173